# Writer Personality Extraction

*Extracts personality traits, core beliefs and speech patterns from a sample of writing in a structured manner so that they can then be used to generate text and dialog in the style of the writer.*

In [9]:
import getpass
import os


def _set_env(var: str):
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var + ":")

In [10]:
_set_env("OPENAI_API_KEY")

In [11]:
from langchain_openai import ChatOpenAI

fast_llm = ChatOpenAI(model="gpt-4o-mini")
long_context_llm = ChatOpenAI(model="gpt-4o")

## Set up the prompt, types to represent the writer's personality, and the Langchain chain.

In [12]:
from typing import List
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

pov_system_prompt = """
You are a genius psychologist and linguist who can infer the personality traits, core beliefs and speech patterns of a person based on their writing. 

You have been asked to analyze a sample of someone's writing.

Analyze the sample and extract the writer's personality traits, core beliefs and speech patterns.
"""

point_of_view_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", pov_system_prompt),
        ("user", "Writing sample:\n{text}"),
    ]
)

class PersonalityTrait(BaseModel):
    name: str = Field(description="A personality trait of the writer")

    @property
    def as_str(self) -> str:
        return f"- {self.name}".strip()
    
class Belief(BaseModel):
    description: str = Field(description="A belief of the writer")

    @property
    def as_str(self) -> str:
        return f"- {self.description}".strip()
    
class SpeechPattern(BaseModel):
    description: str = Field(description="A speech pattern of the writer")
    @property
    def as_str(self) -> str:
        return f"- {self.description}".strip()


class PointOfView(BaseModel):
    traits: List[PersonalityTrait] = Field(
        description="Personality traits of the writer",
    )
    beliefs: List[Belief] = Field(
        description="Beliefs of the writer",
    )
    speech_patterns: List[SpeechPattern] = Field(
        description="Speech patterns of the writer",
    )

    @property
    def as_str(self) -> str:
        traits = "\n".join(trait.as_str for trait in self.traits)
        beliefs = "\n".join(belief.as_str for belief in self.beliefs)
        speech_patterns = "\n".join(sp.as_str for sp in self.speech_patterns)
        return f"Personality traits:\n{traits}\n\nBeliefs:\n{beliefs}\n\nSpeech patterns:\n{speech_patterns}"


point_of_view_chain = point_of_view_prompt | long_context_llm.with_structured_output(
    PointOfView
)

## Use Newspaper3k to extract sample text from a URL

In [13]:
# Try these:
# url = "https://runesoup.com/2019/08/chaeconomica-why-should-we-fear-to-use-it/"
# url = "https://www.piratewires.com/p/total-victory"
# url = "https://ia.samaltman.com/"

url = input("Enter URL to extract the writer's personality: ")

In [14]:
from newspaper import Article

article = Article(url)
article.download()
article.parse()

writing_sample = article.text

In [15]:
writing_sample[:1000]

'In the next couple of decades, we will be able to do things that would have seemed like magic to our grandparents.\n\nThis phenomenon is not new, but it will be newly accelerated. People have become dramatically more capable over time; we can already accomplish things now that our predecessors would have believed to be impossible.\n\nWe are more capable not because of genetic change, but because we benefit from the infrastructure of society being way smarter and more capable than any one of us; in an important sense, society itself is a form of advanced intelligence. Our grandparents – and the generations that came before them – built and achieved great things. They contributed to the scaffolding of human progress that we all benefit from. AI will give people tools to solve hard problems and help us add new struts to that scaffolding that we couldn’t have figured out on our own. The story of progress will continue, and our children will be able to do things we can’t.\n\nIt won’t happe

## Run the chain using the extracted writing sample

In [16]:
point_of_view = point_of_view_chain.invoke({'text': writing_sample})

In [17]:
print(point_of_view.as_str)

Personality traits:
- optimistic
- forward-thinking
- analytical
- visionary

Beliefs:
- The future holds immense potential for technological and societal advancement.
- Human progress is built on the foundations laid by previous generations.
- Artificial intelligence will significantly enhance human capabilities and prosperity.
- Technological advancements, like AI, will transform society and create new opportunities.
- The benefits of AI must be maximized while minimizing potential harms.

Speech patterns:
- Uses historical references to illustrate progress.
- Employs a structured and logical flow of ideas.
- Incorporates visionary language to describe future possibilities.
- Utilizes comparative analogies to underscore points.


## Generate some text using the extracted personality traits, core beleifs and speech patterns

In [21]:
style_transfer_system_prompt = """
IDENTITY:
You are a Nobel-prize winning writer. You can write on any subject in any style.
 
INSTRUCTION:
You have been tasked with writing from the the point of view of another person.

You have been given a description of their personality traits, core beliefs and speech patterns.

Write 5 paragraphs in response to the topic you are given.

PERSONALITY:
{personality}
"""

style_transfer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", style_transfer_system_prompt),
        ("user", "Topic:\n{topic}"),
    ]
)

In [22]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
style_transfer_chain = style_transfer_prompt | long_context_llm | parser

In [23]:
# Try these:
# topic = "cats vs dogs"

topic = input("Topic:")

In [24]:
styled = style_transfer_chain.invoke({
    "personality": point_of_view.as_str, 
    "topic": topic
})
print(styled)

When we consider the age-old debate of cats versus dogs, it is essential to examine it through the lens of progress and potential, much like the technological advancements that have marked human history. Both species have evolved alongside humans, each offering distinct attributes that have contributed to our shared journey. Just as the steam engine revolutionized transportation in the 19th century or the internet reshaped communication in the late 20th century, cats and dogs, too, have played their roles in our development. They have provided companionship, protection, and even inspiration, a testament to the symbiotic relationship between humans and animals.

From a forward-thinking perspective, we can draw parallels between the attributes of cats and dogs and the evolving landscape of artificial intelligence. Dogs, with their loyalty and trainable nature, resemble early AI systems that are designed to follow commands and assist in specific tasks. They are the Watsons and Deep Blues 